Задание 1

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

ratings=pd.read_csv('ml-latest-small/ratings.csv', sep=',')

ratings_count=ratings.groupby('userId').count().reset_index()
ratings_count=ratings_count[['userId','rating']]
ratings_count.head()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [32]:
count_more_than_100=ratings_count[ratings_count['rating']>100]
count_more_than_100.rename(columns={'rating': 'rating_count'}, inplace=True)
count_more_than_100.head()

,userId,rating_count
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [33]:
rating_max=ratings.groupby('userId').max().reset_index()
rating_min=ratings.groupby('userId').min().reset_index()

rating_max_timestamp=rating_max[['userId','timestamp']]
rating_min_timestamp=rating_min[['userId','timestamp']]

rating_max_timestamp.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
rating_min_timestamp.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

rating_max_timestamp.head()

,userId,timestamp_max
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [8]:
rating_min_timestamp.head()

,userId,timestamp_min
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [11]:
avg_timestamp=count_more_than_100.merge(rating_max_timestamp, how='inner').merge(rating_min_timestamp, how='inner')

def average(row):
    mean_=(row['timestamp_max']-row['timestamp_min'])
    return mean_

avg_timestamp['avg']=avg_timestamp.apply(average, axis=1)
len(count_more_than_100)==len(avg_timestamp)

avg_timestamp.head(15)

,userId,rating_count,timestamp_max,timestamp_min,avg
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282
5,21,162,854522908,853157476,1365432
6,22,220,1131753381,1131661890,91491
7,23,726,1166728253,1148386124,18342129
8,26,172,1371811577,1351544316,20267261
9,30,1011,1060795346,944943070,115852276


Задание 2

    Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [20]:
import pandas as pd

zhd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'zhd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
zd

,client_id,zd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [21]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [22]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [23]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [27]:
table=zhd.merge(auto, how='outer', on='client_id')

table=table.merge(air, how='outer', on='client_id')

table.loc[table.zhd_revenue.isnull(), 'zhd_revenue',]=0
table.loc[table.auto_revenue.isnull(), 'auto_revenue',]=0
table.loc[table.air_revenue.isnull(), 'air_revenue',]=0

full_table=table.merge(client_base, how='outer', on='client_id')


full_table

,client_id,zhd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


Задание 3

    В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

    У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
    Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
    Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


In [29]:
import pandas as pd

#Для того что бы добавить координаты нужно ввести индификатор визита так как пользователь может зайти из разных мест
coordinates = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'visit_id': [101, 301, 305, 896, 896],
        'coordinate': ['55.755831 37.617673', '56.755831 37.617673', '55.755831 38.617673', '55.755831 37.617673', '55.255831 37.017673'],
    }
)

coordinates


# Из координат можно получить данные откуда делаются больше всего запросов, что в этой области предпочитают и т. д.


,user_id,visit_id,coordinate
0,11,101,55.755831 37.617673
1,22,301,56.755831 37.617673
2,55,305,55.755831 38.617673
3,11,896,55.755831 37.617673
4,99,896,55.255831 37.017673
